In [1]:
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm
import pickle
import cv2
from math import atan2

In [2]:
th1 = 0.95 #confidence threshold
th2 = 10 #max L-R discrepancy in x coordinate

home_dir = '/data/LiftFly3D/prism/'
data_dir = '/mnt/NAS/SG/prism_data/'
scorer_bottom = 'DLC_resnet50_jointTrackingDec13shuffle1_200000'
scorer_side = 'DLC_resnet50_sideJointTrackingDec17shuffle1_200000'

#joints whose confidence to consider
leg_tips = ['tarsus tip front L', 'tarsus tip mid L', 'tarsus tip back L',
          'tarsus tip front R', 'tarsus tip mid R', 'tarsus tip back R']

crop_positions = ['/bottom_view/videos/crop_loc_191125_PR_Fly1_001_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_002_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_003_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_004_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_001_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_002_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_003_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_004_prism.txt']

videos_side = ['side_view/videos/video_191125_PR_Fly1_001_prism',
               'side_view/videos/video_191125_PR_Fly1_002_prism',
               'side_view/videos/video_191125_PR_Fly1_003_prism',
               'side_view/videos/video_191125_PR_Fly1_004_prism',
               'side_view/videos/video_191125_PR_Fly2_001_prism',
               'side_view/videos/video_191125_PR_Fly2_002_prism',
               'side_view/videos/video_191125_PR_Fly2_003_prism',
               'side_view/videos/video_191125_PR_Fly2_004_prism']

videos_bottom =  ['bottom_view/videos/video_191125_PR_Fly1_001_prism',
                  'bottom_view/videos/video_191125_PR_Fly1_002_prism',
                  'bottom_view/videos/video_191125_PR_Fly1_003_prism',
                  'bottom_view/videos/video_191125_PR_Fly1_004_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_001_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_002_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_003_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_004_prism']

images_side = ['191125_PR/Fly1/001_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly1/002_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly1/003_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly1/004_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly2/001_prism/behData/images/side_view_prism_data_191125_PR_Fly2/',
               '191125_PR/Fly2/002_prism/behData/images/side_view_prism_data_191125_PR_Fly2/',
               '191125_PR/Fly2/003_prism/behData/images/side_view_prism_data_191125_PR_Fly2/',
               '191125_PR/Fly2/004_prism/behData/images/side_view_prism_data_191125_PR_Fly2/']

images_bottom =  ['191125_PR/Fly1/001_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly1/002_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly1/003_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly1/004_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly2/001_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/',
                  '191125_PR/Fly2/002_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/',
                  '191125_PR/Fly2/003_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/',
                  '191125_PR/Fly2/004_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/']

assert len(videos_side)==len(videos_bottom), 'Number of video files must be the same from side and bottom!'

In [3]:
def read_crop_pos(file):
    f=open(file, "r")
    contents =f.readlines()
    pos = []
    for i in range(4,len(contents)):
        line = contents[i][:-1].split(' ')
        pos.append(int(line[1]))
        
    return pos


def orientation(img):
    #_, img_th = cv2.threshold(img, 250, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    img_th = img.copy()
    img_th[img_th < 130] = 0 # was 140
    # Find all the contours in the thresholded image
    contours, _ = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if len(contours) < 1 : return None
    for i, contour in enumerate(contours):
        # Calculate the area of each contour
        area = cv2.contourArea(contour)
        # Ignore contours that are too small or too large
        if area > 10000:
            break

    # Find the orientation of each shape
    img_pts = np.empty((len(contour), 2), dtype=np.float64)
    img_pts[:,0], img_pts[:,1] = contour[:,0,0], contour[:,0,1]

    # PCA analysis
    mean = np.empty((0))
    _, eigenvectors, _ = cv2.PCACompute2(img_pts, mean)

    angle = atan2(eigenvectors[0,1], eigenvectors[0,0])
    return angle


def read_crop_pos(file):
    f=open(file, "r")
    contents =f.readlines()
    im_file = []
    x_pos = []
    for i in range(4,len(contents)):
        line = contents[i][:-1].split(' ')
        im_file.append(line[0])
        x_pos.append(line[1])
        
    return im_file, x_pos


def center_and_align(pts2d, exp):
    '''rotate align data'''
    
    #access corresponding image file
    idx = pts2d.name 
    im_file, _ = read_crop_pos(home_dir + crop_positions[exp])
    im_crop_bottom = cv2.imread(data_dir + images_bottom[exp] + im_file[idx],0)
    
    #get orientation and centre
    angle = orientation(im_crop_bottom)
    c = np.array(im_crop_bottom.shape)/2
    
    #rotate points
    cos, sin = np.cos(angle), np.sin(angle)
    R = np.array(((cos, -sin), (sin, cos)))    
    tmp = pts2d.to_numpy().reshape(-1, 2)
    tmp = np.matmul(tmp-c,R) + c   
    pts2d.iloc[:] = tmp.reshape(-1,tmp.shape[0]*2).flatten()
        
    return pts2d

Filter for high quality frames

In [20]:
index = []
side = pd.DataFrame()
bottom = pd.DataFrame()
for i in tqdm(range(len(videos_side))):
    _side = pd.read_hdf(home_dir + videos_side[i] + scorer_side + '.h5')
    _bottom = pd.read_hdf(home_dir + videos_bottom[i] + scorer_bottom + '.h5')
    
    #drop scorer column label
    _side = _side.droplevel('scorer',axis=1) 
    _bottom = _bottom.droplevel('scorer',axis=1) 
    
    #split L and R (remove if we include flipping)
    side_L_lk = _side.loc[:,(leg_tips[:3],'likelihood')]
    side_R_lk = _side.loc[:,(leg_tips[3:],'likelihood')]
    bottom_lk = _bottom.loc[:,(leg_tips,'likelihood')]
    
    #select for high likelihood frames
    #mask = ((side_L_lk>th1).sum(1)==3) & ((bottom_lk>th1).sum(1)==6) #only flies pointing left
    #mask = ((side_R_lk>th1).sum(1)==3) & ((bottom_lk>th1).sum(1)==6) #only flies pointing right
    mask = ( ((side_L_lk>th1).sum(1)==3) | ((side_R_lk>th1).sum(1)==3) ) & ((bottom_lk>th1).sum(1)==6)
    _side = _side[mask].dropna()
    _bottom = _bottom[mask].dropna()
    
    #sometimes DLC mixes up limbs so take only those frames there the x coordinate matches on side and bottom views
    diff_L = np.abs(_bottom.loc[:,(leg_tips[:3],'x')].values - _side.loc[:,(leg_tips[3:],'x')].values)
    diff_R = np.abs(_bottom.loc[:,(leg_tips[3:],'x')].values - _side.loc[:,(leg_tips[:3],'x')].values)
    mask = ((diff_L<th2).sum(1)==3) | ((diff_R<th2).sum(1)==3)
    _side = _side[mask].dropna()
    _bottom = _bottom[mask].dropna()
    
    #save indices for later
    index.append(_bottom.index.values)
    
    assert _side.shape[0]==_bottom.shape[0], 'Number of rows must match in filtered data!'
    
    #flip left and right side due to prism reflection
    cols = list(_side.columns)
    half = int(len(cols)/2)
    tmp = _bottom.loc[:,cols[:half]].values
    _bottom.loc[:,cols[:half]] = _bottom.loc[:,cols[half:]].values
    _bottom.loc[:,cols[half:]] = tmp
    
    #align horizontally
    _bottom.loc[:,(slice(None),['x','y'])] = _bottom.loc[:,(slice(None),['x','y'])].apply(lambda x: center_and_align(x, i), axis=1)
    
    #rotate flies pointing right
    theta = np.radians(180)
    cos, sin = np.cos(theta), np.sin(theta)
    R = np.array(((cos, -sin), (sin, cos)))
    
    side_R_lk = _side.loc[:,(leg_tips[3:],'likelihood')] #high confidence on R joints means fly points right
    flip_idx = (side_R_lk>th1).sum(1)==3
    
    if np.sum(flip_idx) != 0:
        tmp = _bottom.loc[flip_idx,(slice(None),['x','y'])].to_numpy()
        tmp = np.reshape(tmp, [-1, 2])
        mu = tmp.mean(axis=0)
        tmp = np.matmul(tmp-mu,R) + mu
        tmp = np.reshape( tmp, [-1, 60] )
        _bottom.loc[flip_idx,(slice(None),['x','y'])] = tmp
    
    #also flip left and right side
    #cols = list(_side.columns)
    #half = int(len(cols)/2)
    #tmp = _side.loc[flip_idx,cols[:half]].values
    #_side.loc[flip_idx,cols[:half]] = _side.loc[flip_idx,cols[half:]].values
    #_side.loc[flip_idx,cols[half:]] = tmp
    #tmp = _bottom.loc[flip_idx,cols[:half]].values
    #_bottom.loc[flip_idx,cols[:half]] = _bottom.loc[flip_idx,cols[half:]].values
    #_bottom.loc[flip_idx,cols[half:]] = tmp

    #convert & save to DF3D format
    side_np = _side.loc[:,(slice(None),['x','y'])].to_numpy()
    z = _side.loc[:,(slice(None),'y')].to_numpy()
    side_np = np.stack((side_np[:,::2], side_np[:,1::2]), axis=2)

    bottom_np = _bottom.loc[:,(slice(None),['x','y'])].to_numpy()
    bottom_np = np.stack((bottom_np[:,::2], bottom_np[:,1::2]), axis=2)
    
    poses = {'points2d': np.stack((bottom_np, side_np), axis=0),
             'points3d': np.concatenate((bottom_np, z[:,:,None]), axis=2),
             'index':_side.index.values,
             'flip_idx': np.array(flip_idx),
             #'centroid': c
            }
    
    #save
    pickle.dump(poses,open(home_dir + videos_side[i].split('/')[-1][6:] + '.pkl', 'wb'))

100%|██████████| 8/8 [04:24<00:00, 33.03s/it]


In [ ]:
import matplotlib.pyplot as plt
for i in range(_bottom.shape[0]):
    plt.scatter(poses['points2d'][0,i,:,0],poses['points2d'][0,i,:,1])
    if i==50:
        import sys
        sys.exit()

In [ ]:
plt.scatter(poses['points2d'][0,i,:15,0],poses['points2d'][0,i,:15,1])
plt.scatter(poses['points2d'][0,i,15:,0],poses['points2d'][0,i,15:,1])
plt.scatter(poses['points2d'][0,i,:15,0],275+280-poses['points2d'][0,i,:15,1])

In [19]:
flip_idx

0       False
2       False
3       False
5       False
6       False
        ...  
2538    False
2539    False
2540    False
2542    False
2544    False
Length: 111, dtype: bool

Make joint video

In [ ]:
import subprocess
import cv2
from tqdm import tqdm
    
def imgs_to_video(imgs, fps, out_path):
    '''Write video from a list of images'''
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') #‘F’, ‘M’, ‘P’, ‘4’
    out = cv2.VideoWriter(out_path, fourcc, fps, (imgs[0].shape[1], imgs[0].shape[0]))
                          #imgs[0].shape[:2])
    for img in imgs:
        out.write(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    out.release()
    
                          
def video_to_imgs(vid_path):
    '''Convert video to a list of images'''
    
    cap = cv2.VideoCapture(vid_path)         
    imgs = []            
    while True:
        flag, frame = cap.read()
        if flag:                            
            imgs.append(frame)       
        else:
            break        
    
    return imgs
                                          
    
def concat_vids(vid_name):   
     
    cmd = 'ffmpeg -f concat -safe 0 -i videos.txt -c copy ' + vid_name
    subprocess.call(cmd, shell=True)

    
def videos_to_concat(paths):
    '''
    Make a textfile with paths to videos to concatenate
    '''
    f = open('videos.txt','w+')
    for i in paths:
        f.write("file '" + i + "'\r\n")
        
    f.close()

In [ ]:
print('create videos')
paths = []
for i in tqdm(range(len(videos_side))):       
    #extract frames
    frames = video_to_imgs(home_dir + videos_side[i] + scorer_side + '_labeled.mp4') #convert to frames
    frames = [frames[j] for j in index[i]] #prune frames
    paths.append(home_dir + 'side_' + str(i) + '.mp4') #save output video path for later
    imgs_to_video(frames, 50, paths[i]) #convert back to video
frames = [] #to free up memory 
    
print('concatenate videos')
videos_to_concat(paths)
concat_vids(home_dir + 'side.mp4')
    
print('create videos')    
paths = []
for i in tqdm(range(len(videos_bottom))):    
    #extract frames
    frames = video_to_imgs(home_dir + videos_bottom[i] + scorer_bottom + '_labeled.mp4') #convert to frames
    frames = [frames[j] for j in index[i]] #prune frames
    paths.append(home_dir + 'bottom_' + str(i) + '.mp4') #save output video path for later
    imgs_to_video(frames, 50, paths[i]) #convert back to video

frames = [] #to free up memory    
print('concatenate videos')
videos_to_concat(paths)
concat_vids(home_dir + 'bottom.mp4')    

print('stack videos')
ffmpeg_cmd = "ffmpeg -i " + home_dir + "side.mp4 -i " + home_dir + "bottom.mp4 -filter_complex vstack=inputs=2 " + home_dir + "stacked.mp4"
subprocess.call(ffmpeg_cmd, shell=True)